In [1]:
from vpython import *

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
from scipy.spatial.distance import pdist

In [3]:
dt = 100
R = 1e6
max_mass=5e24
G = 6.67e-11
C=.75

In [4]:

class Ball:
    def __init__(self,mass,position,v_start):
        self.mass = mass
        self.position = position #numpy array with 3 numbers
        self.momentum = v_start*mass
        self.radius = R
        self.sphere = sphere(radius=self.radius,pos=toVec(self.position),color=vec(self.mass/max_mass,1,1))
        
    def update(self,force):
        self.momentum+=force*dt
        self.position+=self.momentum/self.mass*dt
        self.sphere.pos = toVec(self.position)
    
    def __str__(self):
        return str(self.position)
    

In [5]:
def toVec(ar): #ar should just have 3 nums in it
    return vec(ar[0],ar[1],ar[2])

In [6]:
from scipy.spatial.distance import squareform
print(squareform(pdist(np.random.random([4,2]))))

[[0.         0.22361936 0.35259    0.6121414 ]
 [0.22361936 0.         0.39123218 0.38942338]
 [0.35259    0.39123218 0.         0.64061482]
 [0.6121414  0.38942338 0.64061482 0.        ]]


In [7]:
import time
def gravity(balls):
    t = time.time()
    masses = np.array([ball.mass for ball in balls])
    ball_positions = np.array([ball.position for ball in balls])
    num_balls = ball_positions.shape[0]
    dists = squareform(pdist(ball_positions))
    #print(time.time()-t)
    gravities = []
    t = time.time()
    for i in range(num_balls):
        rmat = ball_positions-np.tile(ball_positions[i],[num_balls,1])
        d1 = G*masses*masses[i]/(dists[i]**2+1e-20)
        d2 = rmat/np.tile(np.linalg.norm(rmat,axis=-1)+1e-20,[3,1]).T
        finalmat = np.multiply(np.tile(d1,[3,1]).T,d2)
        gravities.append(np.sum(finalmat,axis=0))
    #print(time.time()-t)
    return gravities

In [18]:
import tensorflow as tf
tf.enable_eager_execution()

In [55]:
import time
tf_G = tf.constant(G,dtype=tf.float64)



@tf.function
def grav2(masses,ball_positions,dists,num_balls):
    gravities = []
    for i in range(num_balls):
        rmat = ball_positions-tf.tile(tf.reshape(ball_positions[i],[1,3]),[num_balls,1])
        d1 = tf_G*masses*masses[i]/(dists[i]**2+1e-20)
        d2 = rmat/tf.transpose(tf.tile(tf.reshape(tf.norm(rmat,axis=-1)+1e-20,[1,num_balls]),[3,1]))
        finalmat = tf.multiply(tf.transpose(tf.tile(tf.reshape(d1,[1,num_balls]),[3,1])),d2)
        gravities.append(tf.reduce_sum(finalmat,axis=0))
    return gravities
def gravity2(balls):
    t = time.time()
    masses = np.array([ball.mass for ball in balls],dtype=np.float64)
    ball_positions = np.array([ball.position for ball in balls])
    num_balls = ball_positions.shape[0]
    dists = squareform(pdist(ball_positions))
    gravities = grav2(masses,ball_positions,dists,num_balls)
    print(time.time()-t)
    return gravities

In [10]:
class Space:
    def __init__(self,balls):
        self.balls = balls
        self.num_balls = len(self.balls)
    def grav_update(self):
        gravities = gravity(self.balls)
        for i,ball in enumerate(self.balls):
            ball.update(gravities[i])
    def collision_update(self):
        ballmat = np.array([ball.position for ball in self.balls])
        dists = squareform(pdist(ballmat))
        np.fill_diagonal(dists,3*R)
        simple = dists-2*R
        colliders = np.nonzero(simple*(simple<0))
        if colliders[0].shape[0]!=0:
            to_add = [np.array([0,0,0],dtype=np.float64) for _ in range(len(self.balls))]
            for i in range(int(colliders[0].shape[0]/2)):
                row,col = colliders[0][i],colliders[1][i]
                ball1,ball2 = self.balls[row],self.balls[col]
                
                rrel = ball2.position-ball1.position
                rrel_hat = rrel/np.linalg.norm(rrel)
                d = 2*R-dists[row][col]
                v1 = ball1.momentum/ball1.mass
                v2 = ball2.momentum/ball2.mass
                vrel = v1-v2
                if np.linalg.norm(vrel)==0:
                    continue
                del_t = d*np.linalg.norm(rrel)/np.dot(vrel,rrel)
                ball1.position-=v1*del_t
                ball2.position-=v2*del_t
                
                vcm = (ball1.momentum+ball2.momentum)/(ball1.mass + ball2.mass)
                v1cm = v1-vcm
                v2cm = v2-vcm

                #really just subtracting 2*projection of v1 onto rrel
                to_add[row] += v1cm-2*np.dot(v1cm,rrel_hat)*rrel_hat
                to_add[col] += v2cm-2*np.dot(v2cm,rrel_hat)*rrel_hat
                
            for add,ball in zip(to_add,self.balls):
                if np.linalg.norm(add)!=0:
                    ball.momentum = add*ball.mass*C
                    ball.position+=dt*(ball.momentum/ball.mass)

In [15]:
import time
balls = [Ball(1e3,np.array([i*3,(i%2)*4,(i%3)*2],dtype=np.float64)*10,np.random.random(3)) for i in range(2000)]
print('generated!')
space = Space(balls)
t = 0
plot = gcurve()
while t<=1000000:
    blech = time.time()
    space.grav_update()
    space.collision_update()
    print(time.time()-blech)
    plot.plot(t,np.linalg.norm(space.balls[0].position-space.balls[1].position))
    t+=dt
    #rate(10000000)

generated!
0.9820401668548584
0.9146089553833008
0.9129617214202881
0.907128095626831
0.8992118835449219
0.9199731349945068
0.8883228302001953
0.9332528114318848
0.9040822982788086
0.8966882228851318
1.1008780002593994
0.9675240516662598
0.9401190280914307
1.0022001266479492


KeyboardInterrupt: 

In [1]:
from vpython import *
import numpy as np
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
dt = 10
R = 1
max_mass=1
G = 6.67e-11
C=1

class Ball:
    def __init__(self,mass,position,v_start):
        self.mass = mass
        self.position = position #numpy array with 3 numbers
        self.momentum = v_start*mass
        self.radius = R
        self.sphere = sphere(radius=self.radius,pos=toVec(self.position),color=vec(self.mass/max_mass,1,1))
        
    def update(self,force):
        self.momentum+=force*dt
        self.position+=self.momentum/self.mass*dt
        self.sphere.pos = toVec(self.position)
    
    def __str__(self):
        return str(self.position)

def toVec(ar): #ar should just have 3 nums in it
    return vec(ar[0],ar[1],ar[2])


import time

maxg = G*max_mass**2/(2*R)**2
def gravity(balls):
    t = time.time()
    masses = np.array([ball.mass for ball in balls],dtype=np.float64)
    ball_positions = np.array([ball.position for ball in balls],dtype=np.float64)
    num_balls = ball_positions.shape[0]
    dists = squareform(pdist(ball_positions))
    #print(time.time()-t)
    gravities = []
    t = time.time()
    for i in range(num_balls):
        gravities.append(np.array([0,0,0],dtype=np.float64))
        for j in range(num_balls):
            if i==j:
                continue
            f = G*masses[i]*masses[j]/dists[i][j]**2
            if f>maxg:
                f = maxg
            gravities[i]+=(ball_positions[j]-ball_positions[i])/np.linalg.norm(ball_positions[j]-ball_positions[i])*f
#     for i in range(num_balls):
#         rmat = ball_positions-np.tile(ball_positions[i],[num_balls,1])
#         d1 = G*masses*masses[i]/(dists[i]**2+1e-100)
#         d2 = rmat/np.tile(np.linalg.norm(rmat,axis=-1)+1e-100,[3,1]).T
#         finalmat = np.multiply(np.tile(d1,[3,1]).T,d2)
#         out = np.sum(finalmat,axis=0)
#         length = np.linalg.norm(out)
#         if length>maxg:
#             out = out/length*maxg
#         gravities.append(out)
    #print(time.time()-t)
    return gravities

class Space:
    def __init__(self,balls):
        self.balls = balls
        self.num_balls = len(self.balls)
    def grav_update(self):
        gravities = gravity(self.balls)
        for i,ball in enumerate(self.balls):
            ball.update(gravities[i])
    def collision_update(self):
        ballmat = np.array([ball.position for ball in self.balls])
        dists = squareform(pdist(ballmat))
        np.fill_diagonal(dists,3*R)
        simple = dists-2*R
        colliders = np.nonzero(simple*(simple<0))
        if colliders[0].shape[0]!=0:
            to_add = [np.array([0,0,0],dtype=np.float64) for _ in range(len(self.balls))]
            for i in range(int(colliders[0].shape[0]/2+1)):
                row,col = colliders[0][i],colliders[1][i]
                ball1,ball2 = self.balls[row],self.balls[col]
                
                rrel = ball2.position-ball1.position
                v1 = ball1.momentum/ball1.mass
                v2 = ball2.momentum/ball2.mass
                vrel = v1-v2
                if np.linalg.norm(vrel)==0:
                    continue
                del_t = 2*R/(np.linalg.norm(rrel+vrel))
                ball1.position-=v1*del_t
                ball2.position-=v2*del_t
                
                rrel = ball2.position-ball1.position
                rrel_hat = rrel/np.linalg.norm(rrel)
                vcm = (ball1.momentum+ball2.momentum)/(ball1.mass + ball2.mass)
                v1cm = v1-vcm
                v2cm = v2-vcm

                #really just subtracting 2*projection of v1 onto rrel
                to_add[row] += v1cm-2*np.dot(v1cm,rrel_hat)*rrel_hat + vcm
                to_add[col] += v2cm-2*np.dot(v2cm,rrel_hat)*rrel_hat + vcm
                
            for add,ball in zip(to_add,self.balls):
                if np.linalg.norm(add)!=0:
                    ball.momentum = add*ball.mass*C
                    ball.position+=dt*(ball.momentum/ball.mass)

<IPython.core.display.Javascript object>

In [2]:
def makePlanet(corner):
    balls = []
    for i in range(2):
        for j in range(2):
            balls.append(Ball(1e24,np.array([i*R*3,j*R*3,0])+corner,np.zeros(3)))
    return balls
def genSpace():
    p1 = makePlanet(np.zeros(3))
    p2 = makePlanet(np.ones(3)*5*R)
    space = Space(p1+p2)
    return space

def genSpace2():
    balls = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                balls.append(Ball(max_mass,np.array([i*R*3,j*R*3,k*R*3],dtype=np.float64),np.zeros(3)))
    return Space(balls)

def genSpace3():
    balls = []
    for i in range(4):
        balls.append(Ball(max_mass,np.array([i*R*3,5,5],dtype=np.float64),np.zeros(3)))
    return Space(balls)
def execute(space):
    start = time.time()
    t = 0
    plot = gcurve()
    count = 0
    while t<=10000000:
        space.grav_update()
        space.collision_update()
        plot.plot(t,space.balls[0].position[0])
        t+=dt
        rate(1000000)
        count+=1
        if (count+1)%1000==0:
            print(count+1)
            print(time.time()-start)
            
dt = 100
C=.5
space = genSpace2()
print('made space!')
execute(space)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

made space!
1000
15.82077693939209
2000
34.76995396614075
3000
50.00481104850769


KeyboardInterrupt: 

In [1]:
from vpython import *

<IPython.core.display.Javascript object>

In [2]:

# Hard-sphere gas.

# Bruce Sherwood

win = 500
CC =.5
G = 6.67e-11

planet_side = 5
Natoms = 2*planet_side**3 # change this to have more or fewer atoms

# Typical values
L = 3 # container is a cube L on a side
gray = color.gray(0.7) # color of edges of container
mass = 1e3 # helium mass
Ratom = 0.03 # wildly exaggerated size of helium atom
k = 1.4E-23 # Boltzmann constant
T = 300 # around room temperature
dt = 1

center1 = vec(-L/4,-L/4,-L/4)
center2 = vec(L/4,L/4,L/4)

animation = canvas( width=win, height=win, align='left')
animation.range = L
animation.title = 'A "hard-sphere" gas'
s = """  Theoretical and averaged speed distributions (meters/sec).
  Initially all atoms have the same speed, but collisions
  change the speeds of the colliding atoms. One of the atoms is
  marked and leaves a trail so you can follow its path.
  
"""
animation.caption = s

d = L/2+Ratom
r = 0.005
boxbottom = curve(color=gray, radius=r)
boxbottom.append([vector(-d,-d,-d), vector(-d,-d,d), vector(d,-d,d), vector(d,-d,-d), vector(-d,-d,-d)])
boxtop = curve(color=gray, radius=r)
boxtop.append([vector(-d,d,-d), vector(-d,d,d), vector(d,d,d), vector(d,d,-d), vector(-d,d,-d)])
vert1 = curve(color=gray, radius=r)
vert2 = curve(color=gray, radius=r)
vert3 = curve(color=gray, radius=r)
vert4 = curve(color=gray, radius=r)
vert1.append([vector(-d,-d,-d), vector(-d,d,-d)])
vert2.append([vector(-d,-d,d), vector(-d,d,d)])
vert3.append([vector(d,-d,d), vector(d,d,d)])
vert4.append([vector(d,-d,-d), vector(d,d,-d)])

Atoms = []
p = []
apos = []
pavg = mass*1e-3 # average kinetic energy p**2/(2mass) = (3/2)kT

gmax = mass**2*G/(2*r)**2

for j1 in range(planet_side):
    for j2 in range(planet_side):
        for j3 in range(planet_side):
            posit = center1 + 3*Ratom*vec(j1,j2,j3)
            Atoms.append(sphere(pos=posit, radius=Ratom, color=gray))
            apos.append(posit)
for j1 in range(planet_side):
    for j2 in range(planet_side):
        for j3 in range(planet_side):
            posit = center2 + 3*Ratom*vec(j1,j2,j3)
            Atoms.append(sphere(pos=posit, radius=Ratom, color=gray))
            apos.append(posit)
#    x = L*random()-L/2
#    y = L*random()-L/2
#    z = L*random()-L/2
#    if i == 0:
#        Atoms.append(sphere(pos=vector(x,y,z), radius=Ratom, color=color.cyan, make_trail=True, retain=100, trail_radius=0.3*Ratom))
#    else: Atoms.append(sphere(pos=vector(x,y,z), radius=Ratom, color=gray))
#    apos.append(vec(x,y,z))
#    theta = pi*random()
#    phi = 2*pi*random()
#    px = pavg*sin(theta)*cos(phi)
#    py = pavg*sin(theta)*sin(phi)
#    pz = pavg*cos(theta)
#    p.append(vector(px,py,pz))
for i in range(Natoms):
    p.append(vector(0,0,0))

deltav = 100 # binning for v histogram

def barx(v):
    return int(v/deltav) # index into bars array

nhisto = int(4500/deltav)
histo = []
for i in range(nhisto): histo.append(0.0)
histo[barx(pavg/mass)] = Natoms

gg = graph( width=win, height=0.4*win, xmax=3000, align='left',
    xtitle='speed, m/s', ytitle='Number of atoms', ymax=Natoms*deltav/1000)

theory = gcurve( color=color.cyan )
dv = 10
for v in range(0,3001+dv,dv):  # theoretical prediction
    theory.plot( v, (deltav/dv)*Natoms*4*pi*((mass/(2*pi*k*T))**1.5) *exp(-0.5*mass*(v**2)/(k*T))*(v**2)*dv )

accum = []
for i in range(int(3000/deltav)): accum.append([deltav*(i+.5),0])
vdist = gvbars(color=color.red, delta=deltav )

def interchange(v1, v2):  # remove from v1 bar, add to v2 bar
    barx1 = barx(v1)
    barx2 = barx(v2)
    if barx1 == barx2:  return
    if barx1 >= len(histo) or barx2 >= len(histo): return
    histo[barx1] -= 1
    histo[barx2] += 1
    
def checkCollisions():
    hitlist = []
    r2 = 2*Ratom
    r2 *= r2
    for i in range(Natoms):
        ai = apos[i]
        for j in range(i) :
            aj = apos[j]
            dr = ai - aj
            if mag2(dr) < r2: hitlist.append([i,j])
    return hitlist

nhisto = 0 # number of histogram snapshots to average

while True:
    rate(300)
    # Accumulate and average histogram snapshots
    for i in range(len(accum)): accum[i][1] = (nhisto*accum[i][1] + histo[i])/(nhisto+1)
    if nhisto % 10 == 0:
        vdist.data = accum
    nhisto += 1

    # Update all positions
    for i in range(Natoms): Atoms[i].pos = apos[i] = apos[i] + (p[i]/mass)*dt
    
    for i in range(Natoms):
        for j in range(Natoms):
            if i==j:
                continue
            atomi = apos[i]
            atomj = apos[j]
            if mag2(atomi-atomj)==0:
                print('oops')
                print(i)
                print(j)
                break
            f = G*mass**2/mag2(atomi-atomj)*norm(atomi-atomj)
            if mag(f)>gmax:
                f = norm(f)*gmax
            p[j]+=f*dt
            
    
    # Check for collisions
    hitlist = checkCollisions()

    # If any collisions took place, update momenta of the two atoms
    for ij in hitlist:
        i = ij[0]
        j = ij[1]
        ptot = p[i]+p[j]
        posi = apos[i]
        posj = apos[j]
        vi = p[i]/mass
        vj = p[j]/mass
        vrel = vj-vi

        a = vrel.mag2
        if a == 0: continue;  # exactly same velocities
        rrel = posi-posj
        if rrel.mag > Ratom:
            continue # one atom went all the way through another
    
        # theta is the angle between vrel and rrel:
        dx = dot(rrel, vrel.hat)       # rrel.mag*cos(theta)
        dy = cross(rrel, vrel.hat).mag # rrel.mag*sin(theta)
        # alpha is the angle of the triangle composed of rrel, path of atom j, and a line
        #   from the center of atom i to the center of atom j where atome j hits atom i:
        alpha = asin(dy/(2*Ratom)) 
        d = (2*Ratom)*cos(alpha)-dx # distance traveled into the atom from first contact
        deltat = d/vrel.mag         # time spent moving from first contact to position inside atom
        posi = posi-vi*deltat # back up to contact configuration
        posj = posj-vj*deltat
        mtot = 2*mass
        pcmi = p[i]-ptot*mass/mtot # transform momenta to cm frame
        pcmj = p[j]-ptot*mass/mtot
        rrel = norm(rrel)
        pcmi = (pcmi-2*pcmi.dot(rrel)*rrel)*CC # bounce in cm frame
        pcmj = (pcmj-2*pcmj.dot(rrel)*rrel)*CC
        p[i] = pcmi+ptot*mass/mtot # transform momenta back to lab frame
        p[j] = pcmj+ptot*mass/mtot
        apos[i] = posi+(p[i]/mass)*deltat # move forward deltat in time
        apos[j] = posj+(p[j]/mass)*deltat
        interchange(vi.mag, p[i].mag/mass)
        interchange(vj.mag, p[j].mag/mass)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 